# Learn the basics of the Druid API

<!--
  ~ Licensed to the Apache Software Foundation (ASF) under one
  ~ or more contributor license agreements.  See the NOTICE file
  ~ distributed with this work for additional information
  ~ regarding copyright ownership.  The ASF licenses this file
  ~ to you under the Apache License, Version 2.0 (the
  ~ "License"); you may not use this file except in compliance
  ~ with the License.  You may obtain a copy of the License at
  ~
  ~   http://www.apache.org/licenses/LICENSE-2.0
  ~
  ~ Unless required by applicable law or agreed to in writing,
  ~ software distributed under the License is distributed on an
  ~ "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY
  ~ KIND, either express or implied.  See the License for the
  ~ specific language governing permissions and limitations
  ~ under the License.
  -->
  
This tutorial introduces you to the basics of the Druid API and some of the endpoints you might use frequently to perform tasks, such as the following:

- Checking if your cluster is up
- Ingesting data
- Querying data

Different [Druid server types](https://druid.apache.org/docs/latest/design/processes.html#server-types)  are responsible for handling different APIs for the Druid services. For example, the Overlord service on the Master server provides the status of a task. You'll also interact the Broker service on the Query Server to see what datasources are available. And to run queries, you'll interact with the Broker. The Router service on the Query servers routes API calls.

For more information, see the [API reference](https://druid.apache.org/docs/latest/operations/api-reference.html), which is organized by server type.

For work within other notebooks, prefer to use the [Python API](Python_API_Tutorial.ipynb) which is a notebook-friendly wrapper around the low-level API calls shown here.

## Table of contents

- [Prerequisites](#Prerequisites)
- [Get basic cluster information](#Get-basic-cluster-information)
- [Ingest data](#Ingest-data)
- [Query your data](#Query-your-data)
- [Learn more](#Learn-more)

For the best experience, use JupyterLab so that you can always access the table of contents.

## Prerequisites

Install the [Requests](https://requests.readthedocs.io/en/latest/) library for Python before you start. For example:

```bash
pip install requests
```

Please read the [Requests Quickstart](https://requests.readthedocs.io/en/latest/user/quickstart/) to gain a basic understanding of how Requests works.

Next, you'll need a Druid cluster. This tutorial uses the `micro-quickstart` config described in the [Druid quickstart](https://druid.apache.org/docs/latest/tutorials/index.html). So download that and start it if you haven't already. In the root of the Druid folder, run the following command to start Druid:

```bash
./bin/start-micro-quickstart
```

Finally, you'll need either JupyterLab (recommended) or Jupyter Notebook. Both the quickstart Druid cluster and Jupyter are deployed at `localhost:8888` by default, so you'll need to change the port for Jupyter. To do so, stop Jupyter and start it again with the `port` parameter included. For example, you can use the following command to start Jupyter on port `3001`:

```bash
# If you're using JupyterLab
jupyter lab --port 3001
# If you're using Jupyter Notebook
jupyter notebook --port 3001 
```

To start this tutorial, run the next cell. It imports the Python packages you'll need and defines a variable for the the Druid host, where the Router service listens.

`druid_host` is the hostname and port for your Druid deployment. In a distributed environment, you can point to other Druid services. In this tutorial, you'll use the Router service as the `druid_host`.

In [ ]:
import requests

druid_host = "http://localhost:8888"
druid_host

If your cluster is secure, you'll need to provide authorization information on each request. You can automate it by using the Requests `session` feature. Although this tutorial assumes no authorization, the configuration below defines a session as an example.

In [ ]:
session = requests.Session()

In the rest of this tutorial, the `endpoint` and other variables are updated in code cells to call a different Druid endpoint to accomplish a task.

## Get basic cluster information

In this cell, you'll use the `GET /status` endpoint to return basic information about your cluster, such as the Druid version, loaded extensions, and resource consumption.

The following cell sets `endpoint` to `/status` and updates the HTTP method to `GET`. 

In [ ]:
endpoint = druid_host + '/status'
endpoint

The Requests `Session` has a `get()` method that posts an HTTP `GET` request. The method takes multiple arguments. Here you only need the URL. The method returns a Requests `Response` object, which can convert the returned JSON result to Python. JSON objects become Python dictionaries. JSON arrays become Python arrays. When you run the cell, you should receive a response that starts with the version number of your Druid deployment.

In [ ]:
response = session.get(endpoint)
json = response.json()
json

Because the JSON result is converted to Python, you can use Python to pull out the information you want. For example, to see just the version:

In [ ]:
json['version']

### Get cluster health

The `/status/health` endpoint returns JSON `true` if your cluster is up and running. It's useful if you want to do something like programmatically check if your cluster is available. When you run the following cell, you should receive the `True` Python value if your Druid cluster has finished starting up and is running.

In [ ]:
endpoint = druid_host + '/status/health'
endpoint

In [ ]:
is_healthy = session.get(endpoint).json()
is_healthy

## Ingest data

Now that you've confirmed that your cluster is up and running, you can start ingesting data. There are different ways to ingest data based on what your needs are. For more information, see [Ingestion methods](https://druid.apache.org/docs/latest/ingestion/index.html#ingestion-methods).

This tutorial uses the multi-stage query (MSQ) task engine and its `sql/task` endpoint to perform SQL-based ingestion.

To learn more about SQL-based ingestion, see [SQL-based ingestion](https://druid.apache.org/docs/latest/multi-stage-query/index.html). For information about the endpoint specifically, see [SQL-based ingestion and multi-stage query task API](https://druid.apache.org/docs/latest/multi-stage-query/api.html).

In [ ]:
endpoint = druid_host + '/druid/v2/sql/task'
endpoint

The example uses INSERT, but you could also use REPLACE INTO. In fact, if you have an existing datasource with the name `wikipedia_api`, you need to use REPLACE INTO instead. 

The `/sql/task` endpoint accepts [SQL requests in the JSON-over-HTTP format](https://druid.apache.org/docs/latest/querying/sql-api.html#request-body) using the `query`, `context`, and `parameters` fields.

The query inserts data from an external source into a table named `wikipedia_api`. 

Before you ingest the data, take a look at the query. Pay attention to two parts of it, `__time` and `PARTITIONED BY`, which relate to how Druid partitions data:

-  **`__time`**

   The `__time` column is a key concept for Druid. It's the default partition for Druid and is treated as the primary timestamp. Use it to help you write faster and more efficient queries. Big datasets, such as those for event data, typically have a time component. This means that instead of writing a query using only `COUNT`, you can combine that with `WHERE __time` to return results much more quickly.

-  **`PARTITIONED BY DAY`**

   If you partition by day, Druid creates segment files within the partition based on the day. You can only replace, delete and update data at the partition level. So when you're deciding how to partition data, make the partition large enough (min 500,000 rows) for good performance but not so big that those operations become impractical to run.

To learn more, see [Partitioning](https://druid.apache.org/docs/latest/ingestion/partitioning.html).

The query uses `INSERT INTO`. If you have an existing datasource with the name `wikipedia_api`, use `REPLACE INTO` instead.

In [ ]:
sql = '''
INSERT INTO wikipedia_api 
SELECT 
  TIME_PARSE("timestamp") AS __time,
  * 
FROM TABLE(EXTERN(
  '{"type": "http", "uris": ["https://druid.apache.org/data/wikipedia.json.gz"]}', 
  '{"type": "json"}', 
  '[{"name": "added", "type": "long"}, {"name": "channel", "type": "string"}, {"name": "cityName", "type": "string"}, {"name": "comment", "type": "string"}, {"name": "commentLength", "type": "long"}, {"name": "countryIsoCode", "type": "string"}, {"name": "countryName", "type": "string"}, {"name": "deleted", "type": "long"}, {"name": "delta", "type": "long"}, {"name": "deltaBucket", "type": "string"}, {"name": "diffUrl", "type": "string"}, {"name": "flags", "type": "string"}, {"name": "isAnonymous", "type": "string"}, {"name": "isMinor", "type": "string"}, {"name": "isNew", "type": "string"}, {"name": "isRobot", "type": "string"}, {"name": "isUnpatrolled", "type": "string"}, {"name": "metroCode", "type": "string"}, {"name": "namespace", "type": "string"}, {"name": "page", "type": "string"}, {"name": "regionIsoCode", "type": "string"}, {"name": "regionName", "type": "string"}, {"name": "timestamp", "type": "string"}, {"name": "user", "type": "string"}]'
  ))
PARTITIONED BY DAY
'''

The query is included inline here. You can also store it in a file and provide the file.

The above showed how the Requests library can convert the response from JSON to Python. Requests can also convert the request from Python to JSON. The next cell builds up a Python map that represents the Druid `SqlRequest` object. In this case, you need the query and a context variable to set the task count to 3.

In [ ]:
sql_request = {
    'query': sql,
    'context': {
      'maxNumTasks': 3
  }
}

With the SQL request ready, use the the `json` parameter to the `Session` `post` method to send a `POST` request with the `sql_request` object as the payload. The result is a Requests `Response` which is saved in a variable.

Now, run the next cell to start the ingestion.

In [ ]:
response = session.post(endpoint, json=sql_request)

The MSQ task engine uses a task to ingest data. The response for the API includes a `taskId` and `state` for your ingestion. You can use this `taskId` to reference this task later on to get more information about it.

It is good practice to ensure that the response succeeded by checking the return status. The status should be 20x. (202 means "accepted".) If the response is something else, such as 4xx, display `response.text` to see the error message.

In [ ]:
response.status_code

Convert the JSON response to a Python object.

In [ ]:
json = response.json()
json

Extract the taskId value from the taskId_response variable so that you can reference it later:

In [ ]:
ingestion_taskId = json['taskId']
ingestion_taskId

### Get the status of your task

The following cell shows you how to get the status of your ingestion task. The example continues to run API calls against the endpoint to fetch the status until the ingestion task completes. When it's done, you'll see the JSON response.

You can see basic information about your query, such as when it started and whether or not it's finished.

In addition to the status, you can retrieve a full report about it if you want using `GET /druid/indexer/v1/task/TASK_ID/reports`. But you won't need that information for this tutorial.

In [ ]:
endpoint = druid_host + f"/druid/indexer/v1/task/{ingestion_taskId}/status"
endpoint

In [ ]:
json = session.get(endpoint).json()
json

Wait until your ingestion completes before proceeding. Depending on what else is happening in your Druid cluster and the resources available, ingestion can take some time. Pro tip: an asterisk appears next to the cell while Python runs and waits for the task.

In [ ]:
import time

ingestion_status = json['status']['status']

if ingestion_status == "RUNNING":
  print("The ingestion is running...")

while ingestion_status != "SUCCESS":
  time.sleep(5) # 5 seconds 
  json = session.get(endpoint).json()
  ingestion_status = json['status']['status']
  
if ingestion_status == "SUCCESS": 
  print("The ingestion is complete")
else:
  print("The ingestion task failed:", json)

## Query your data

When you ingest data into Druid, Druid stores the data in a datasource, and this datasource is what you run queries against.

### List your datasources

You can get a list of datasources from the `/druid/coordinator/v1/datasources` endpoint. Since you're just getting started, there should only be a single datasource, the `wikipedia_api` table you created earlier when you ingested external data.

In [ ]:
endpoint = druid_host + '/druid/coordinator/v1/datasources'
endpoint

In [ ]:
session.get(endpoint).json()

### SELECT data

Now, you can query the data. Because this tutorial is running in Jupyter, make sure to limit the size of your query results using `LIMIT`. For example, the following cell selects all columns but limits the results to 3 rows for display purposes because each row is a JSON object. In actual use cases, you'll want to only select the rows that you need. For more information about the kinds of things you can do, see [Druid SQL](https://druid.apache.org/docs/latest/querying/sql.html).


In [ ]:
endpoint = druid_host + '/druid/v2/sql'
endpoint

As for ingestion, define a query, then create a `SQLRequest` object as a Python `dict`.

In [ ]:
sql = '''
SELECT  * 
FROM wikipedia_api 
LIMIT 3
'''
sql_request = {'query': sql}

Now run the query. The result is an array of JSON objects. Each JSON object in the response represents a row in the `wikipedia_api` datasource.

In [ ]:
json = session.post(endpoint, json=sql_request).json()
json

In addition to the query, there are a few additional things you can define within the payload. For a full list, see [Druid SQL API](https://druid.apache.org/docs/latest/querying/sql-api.html)

This tutorial uses a context parameter and a dynamic parameter.

Context parameters can control certain characteristics related to a query, such as configuring a custom timeout. For information, see [Context parameters](https://druid.apache.org/docs/latest/querying/query-context.html). In the example query that follows, the context block assigns a custom `sqlQueryID` to the query. Typically, the `sqlQueryId` is autogenerated. With a custom ID, you can use it to reference the query more easily. For example, if you need to cancel a query.


Druid supports dynamic parameters, so you can either define certain parameters within the query explicitly or insert a `?` as a placeholder and define it in a parameters block. In the following cell, the `?` gets bound to the timestmap value of `2016-06-27` at execution time. For more information, see [Dynamic parameters](https://druid.apache.org/docs/latest/querying/sql.html#dynamic-parameters).


The following cell selects rows where the `__time` column contains a value greater than the value defined dynamically in `parameters` and sets a custom `sqlQueryId`.

In [ ]:
sql = '''
SELECT * 
FROM wikipedia_api 
WHERE __time > ? 
LIMIT 1
'''
sql_request = {
    'query': sql,
    'context': {
      'sqlQueryId' : 'important-query'
    },
    'parameters': [
      { 'type': 'TIMESTAMP', 'value': '2016-06-27'}
  ]
}

In [ ]:
json = session.post(endpoint, json=sql_request).json()
json

## Learn more

This tutorial covers the some of the basics related to the Druid API. To learn more about the kinds of things you can do, see the API documentation:

- [Druid SQL API](https://druid.apache.org/docs/latest/querying/sql-api.html)
- [API reference](https://druid.apache.org/docs/latest/operations/api-reference.html)

You can also try out the [druid-client](https://github.com/paul-rogers/druid-client), a Python library for Druid created by Paul Rogers, a Druid contributor. A simplified version of that library is included with these tutorials. See [the Python API Tutorial](Python_API_Tutorial.ipynb) for an overview. That tutorial shows how to do the same tasks as this one, but in a simpler form: focusing on the Druid actions and not the mechanics of the REST API.